In [2]:
import argparse
import os
import timeit
from pprint import pprint
import joblib
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from xgboost.sklearn import XGBClassifier
from feast import FeatureStore

In [ ]:
label_path = r"/home/hjhwang/workspace/tpcx-ai-v1.0.3.1/labels.parquet"
label_data = pd.read_parquet(label_path)
label_data = label_data[(label_data['event_timestamp'] >= '2024-01-01') & (label_data['event_timestamp'] < '2024-05-01')]

In [ ]:
store = FeatureStore(repo_path="/home/hjhwang/workspace/tpcx-ai-v1.0.3.1/feast_uc08/feature_repo")

## Generating Training Data
- (1) Get historical feature from offline store
- (2) Convert to pandas dataframe

In [43]:
start = timeit.default_timer()
training_df = store.get_historical_features(
    entity_df=label_data ,
features = [
    "line_order_product:scan_count",
    "line_order_product:scan_count_abs",
    # "line_order_product:Monday",
    # "line_order_product:Tuesday",
    # "line_order_product:Wednesday",
    # "line_order_product:Thursday",
    # "line_order_product:Friday",
    # "line_order_product:Saturday",
    # "line_order_product:Sunday",
    # "line_order_product:FINANCIAL SERVICES",
    # "line_order_product:SHOES",
    # "line_order_product:PERSONAL CARE",
    # "line_order_product:PAINT AND ACCESSORIES",
    # "line_order_product:DSD GROCERY",
    # "line_order_product:MEAT - FRESH & FROZEN",
    # "line_order_product:DAIRY",
    # "line_order_product:PETS AND SUPPLIES",
    # "line_order_product:HOUSEHOLD CHEMICALS/SUPP",
    # "line_order_product:IMPULSE MERCHANDISE",
    # "line_order_product:PRODUCE",
    # "line_order_product:CANDY, TOBACCO, COOKIES",
    # "line_order_product:GROCERY DRY GOODS",
    # "line_order_product:BOYS WEAR",
    # "line_order_product:FABRICS AND CRAFTS",
    # "line_order_product:JEWELRY AND SUNGLASSES",
    # "line_order_product:MENS WEAR",
    # "line_order_product:ACCESSORIES",
    # "line_order_product:HOME MANAGEMENT",
    # "line_order_product:FROZEN FOODS",
    # "line_order_product:SERVICE DELI",
    # "line_order_product:INFANT CONSUMABLE HARDLINES",
    # "line_order_product:PRE PACKED DELI",
    # "line_order_product:COOK AND DINE",
    # "line_order_product:PHARMACY OTC",
    # "line_order_product:LADIESWEAR",
    # "line_order_product:COMM BREAD",
    # "line_order_product:BAKERY",
    # "line_order_product:HOUSEHOLD PAPER GOODS",
    # "line_order_product:CELEBRATION",
    # "line_order_product:HARDWARE",
    # "line_order_product:BEAUTY",
    # "line_order_product:AUTOMOTIVE",
    # "line_order_product:BOOKS AND MAGAZINES",
    # "line_order_product:SEAFOOD",
    # "line_order_product:OFFICE SUPPLIES",
    # "line_order_product:LAWN AND GARDEN",
    # "line_order_product:SHEER HOSIERY",
    # "line_order_product:WIRELESS",
    # "line_order_product:BEDDING",
    # "line_order_product:BATH AND SHOWER",
    # "line_order_product:HORTICULTURE AND ACCESS",
    # "line_order_product:HOME DECOR",
    # "line_order_product:TOYS",
    # "line_order_product:INFANT APPAREL",
    # "line_order_product:LADIES SOCKS",
    # "line_order_product:PLUS AND MATERNITY",
    # "line_order_product:ELECTRONICS",
    # "line_order_product:GIRLS WEAR, 4-6X  AND 7-14",
    # "line_order_product:BRAS & SHAPEWEAR",
    # "line_order_product:LIQUOR,WINE,BEER",
    # "line_order_product:SLEEPWEAR/FOUNDATIONS",
    # "line_order_product:CAMERAS AND SUPPLIES",
    # "line_order_product:SPORTING GOODS",
    # "line_order_product:PLAYERS AND ELECTRONICS",
    # "line_order_product:PHARMACY RX",
    # "line_order_product:MENSWEAR",
    # "line_order_product:OPTICAL - FRAMES",
    # "line_order_product:SWIMWEAR/OUTERWEAR",
    # "line_order_product:OTHER DEPARTMENTS",
    # "line_order_product:MEDIA AND GAMING",
    # "line_order_product:FURNITURE",
    # "line_order_product:OPTICAL - LENSES",
    # "line_order_product:SEASONAL",
    # "line_order_product:LARGE HOUSEHOLD GOODS",
    # "line_order_product:1-HR PHOTO",
    # "line_order_product:CONCEPT STORES",
    # "line_order_product:HEALTH AND BEAUTY AIDS"
]
  ,
).to_df()
end = timeit.default_timer()
pre_process_time = end - start
print('generating train data time:\t', pre_process_time)

/home/hjhwang/workspace/tpcx-ai-v1.0.3.1/lib/python-venv/lib/python3.9/site-packages/feast/infra/offline_stores/file.py:489: UserWarning: Discarding nonzero nanoseconds in conversion.
  entity_df_event_timestamp.min().to_pydatetime(),
/home/hjhwang/workspace/tpcx-ai-v1.0.3.1/lib/python-venv/lib/python3.9/site-packages/feast/infra/offline_stores/file.py:490: UserWarning: Discarding nonzero nanoseconds in conversion.
  entity_df_event_timestamp.max().to_pydatetime(),


generating train data time:	 12.817504856735468


In [44]:
training_df.describe


<bound method NDFrame.describe of          o_order_id  trip_type                     event_timestamp  \
0           2421255         21 2024-01-01 00:00:01.507199232+00:00   
1            351084         14 2024-01-01 00:00:03.639835648+00:00   
2           1278572          8 2024-01-01 00:00:04.277054208+00:00   
3            391200         13 2024-01-01 00:00:09.295401728+00:00   
4           3280142         34 2024-01-01 00:00:14.825527040+00:00   
...             ...        ...                                 ...   
2411603     2190868         23 2024-04-30 23:59:34.864157696+00:00   
2411604     1312069          1 2024-04-30 23:59:46.005153024+00:00   
2411605     2863779         21 2024-04-30 23:59:48.816577280+00:00   
2411606      300712         35 2024-04-30 23:59:54.730125056+00:00   
2411607     1755610         18 2024-04-30 23:59:59.703918080+00:00   

         scan_count  scan_count_abs  
0                10              10  
1                21              21  
2          

## Generating Inference Data
- (1) Get Online features

In [40]:
!cd /home/hjhwang/workspace/tpcx-ai-v1.0.3.1/feast_uc08/feature_repo && feast materialize 2024-05-01T00:00:00+00:00 2024-05-31T00:00:00+00:00

07/03/2024 02:51:31 PM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
Materializing 1 feature views from 2024-05-01 09:00:00+09:00 to 2024-05-31 09:00:00+09:00 into the sqlite online store.

line_order_product:
  0%|                                                                    | 0/596084 [00:00<?, ?it/s]/home/hjhwang/workspace/tpcx-ai-v1.0.3.1/lib/python-venv/lib/python3.9/site-packages/feast/utils.py:224: UserWarning: Discarding nonzero nanoseconds in conversion.
  return ts.to_pydatetime()
  2%|▉                                                       | 9923/596084 [00:11<10:10, 959.41it/s]/home/hjhwang/workspace/tpcx-ai-v1.0.3.1/lib/python-venv/lib/python3.9/site-packages/feast/utils.py:224: UserWarning: Discarding nonzero nanoseconds in conversion.
  return ts.to_pydatetime()
  3%|█▊                                                     | 19979/

In [48]:
start = timeit.default_timer()

feature_vector = store.get_online_features(
    features=[
        "line_order_product:scan_count",
        "line_order_product:scan_count_abs",
    ],
    entity_rows=[
        {"o_order_id":10},
        {"o_order_id":12},
        {"o_order_id":26},
        {"o_order_id":27},
        {"o_order_id":32},
        {"o_order_id":36},
        {"o_order_id":50},
        {"o_order_id":53},
        {"o_order_id":55},   
    ]
).to_dict()
end = timeit.default_timer()
pre_process_time = end - start
print('generating inference data time:\t', pre_process_time)

{'o_order_id': [10, 12, 26, 27, 32, 36, 50, 53, 55], 'scan_count': [21, 6, 3, 26, 14, 10, 25, 10, 27], 'scan_count_abs': [21, 6, 3, 26, 14, 10, 25, 10, 27]}
generating inference data time:	 0.005055591464042664
